In [1]:
# Libraries
from ismn.interface import ISMN_Interface
import numpy as np
import matplotlib.pyplot as plt 

In [3]:
# Either a .zip file or one folder that contains all networks, here we read from .zip
data_path = "../data/raw/ISMN_2012_2020.zip"
ismn_data = ISMN_Interface(data_path)

Processing metadata for all ismn stations into folder ..\data\raw\ISMN_2012_2020.zip.
This may take a few minutes, but is only done once...
Hint: Use `parallel=True` to speed up metadata generation for large datasets


Files Processed: 0it [00:00, ?it/s]

Metadata generation finished after 0 Seconds.
Metadata and Log stored in ..\data\raw\python_metadata


ValueError: No objects to concatenate